In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

import neuro_morpho_toolbox as nmt
ns = nmt.neuron_set('/home/penglab/Documents/Janelia_1000')
_ = ns.ReduceDimUMAP(feature_set="projection_features", n_neighbors=100)


/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/
Loading CCF Atlas data...
Loading time: 0.81
Loading CCF brain structure data...
Loading time: 0.00
Loading...
100 loaded: 2.5s
Invalid number of soma found: 0
QC failed: AA0114.swc
Invalid number of soma found: 0
QC failed: AA0115.swc
200 loaded: 2.2s
300 loaded: 2.3s
400 loaded: 2.4s
Invalid number of soma found: 0
QC failed: AA0472.swc
500 loaded: 2.3s
Invalid number of soma found: 0
QC failed: AA0576.swc
Invalid number of soma found: 0
QC failed: AA0585.swc
Invalid number of soma found: 0
QC failed: AA0589.swc
600 loaded: 2.1s
Invalid number of soma found: 0
QC failed: AA0639.swc
Invalid number of soma found: 0
QC failed: AA0670.swc
Invalid number of soma found: 0
QC failed: AA0672.swc
700 loaded: 2.1s
Invalid number of soma found: 0
QC failed: AA0754.swc
Invalid number of soma found: 0
QC failed: AA0763.swc
800 loaded: 2.6s
900 loaded: 1.8s
QC failed: AA0952.swc
QC failed: AA0964.swc
QC failed: AA0968.swc
QC

/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/neuron_features.py:337: RuntimeWarning:

divide by zero encountered in true_divide

/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/neuron_features.py:337: RuntimeWarning:

invalid value encountered in true_divide




## Using axon projection as features
SETTING CLUSTER NUMBER FROM 8 TO 40

### Hierarchy Clustering
For Hierarchy method
* the most suitable parameter is {'L_method': 'centroid', 'L_metric': 'braycurtis', 'criterionH': 'distance', 'depth': 2, 'R': None, 't': 0.9, 'optimal_ordering': False}
* the ARI is 0.258210445492697
* The setting cluster number's limit is satisfied, the final number of cluster is 19

### Kmeans Clustering
For Kmeans method
* the most suitable parameter is {'n_clusters': 14, 'init': 'random', 'n_init': 20, 'max_iter': 300, 'tol': 0.0001, 'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 'algorithm': 'full'}
* the ARI is 0.2129613044735105
* The setting cluster number's limit is satisfied, the final number of cluster is 14

### DBSCAN Clustering
For DBSCAN method
* the most suitable parameter is {'eps': 0.215, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 'leaf_size': 30, 'p': None, 'n_jobs': None}
* the ARI is 0.2812503177620878
* The setting cluster number's limit is satisfied, the final number of cluster is 24

### HDBSCAN Clustering
For HDBSCAN method, 
* the most suitable parameter is {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.8, 'min_samples': 7, 'p': 2, 'algorithm': 'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 'prediction_data': False, 'match_reference_implementation': False}
* the ARI is 0.28287214992477994
* The setting cluster number's limit is satisfied, the final number of cluster is 31

### SNN Clustering
For SNN_community method
* the most suitable parameter is {'knn': 5, 'metric': 'minkowski', 'method': 'FastGreedy'}
* the ARI is 0.22710873358441527
* The setting cluster number's limit is satisfied, the final number of cluster is 31

In [2]:
par_hier =  {'L_method': 'centroid', 'L_metric': 'euclidean', 'criterionH': 'distance', 'depth': 2, 'R': None,
             't': 0.9, 'optimal_ordering': False}
par_kmeans = {'n_clusters': 14, 'init': 'random', 'n_init': 20, 'max_iter': 300, 'tol': 0.0001, 
              'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None,
              'algorithm': 'full'}

par_dbscan = {'eps': 0.215, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 
              'leaf_size': 30, 'p': None, 'n_jobs': None}

par_hdbscan = {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.8, 'min_samples': 7, 'p': 2, 'algorithm': 
               'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 
               'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 
               'prediction_data': False, 'match_reference_implementation': False}

par_snn = {'knn':5,'metric':'minkowski','method':'FastGreedy'}


In [3]:
def get_clusters(inputUMAP,method='SNN_community',karg_dict={'knn':5, 'metric':'minkowski','method':'FastGreedy'}):
    methods_allowed = ['SNN_community', 'Hierarchy', 'Kmeans', 'DBSCAN', 'HDBSCAN']
    assert method in methods_allowed, "Please set 'method' as one of the following: 'SNN_community', 'Hierarchy', 'Kmeans', 'DBSCAN', 'HDBSCAN'"
    selectedUMAP = inputUMAP.copy()
    if method=='SNN_community':
        #print('Result of SNN_community')
        if 'knn' in karg_dict.keys():
            knn = karg_dict['knn']
        else:
            knn = 5
        if 'metric' in karg_dict.keys():
            metric = karg_dict['metric']
        else:
            metric = 'minkowski'
        if 'method' in karg_dict.keys():
            community_method = karg_dict['method']
        else:
            community_method = 'FastGreedy'
        cur_clusters = nmt.get_clusters_SNN_community(selectedUMAP, knn=knn, metric=metric, method=community_method)
        

    #karg_dict={'L_method':'single','L_metric':'euclidean'.'t':0.9,'criterionH':'inconsistent', depth=2, R=None, monocrit=None}
    if method =='Hierarchy':
        #print('Result of Hierarchy CLustering')
        cur_clusters = nmt.get_clusters_Hierarchy_clustering(selectedUMAP, karg_dict)


    if method =='Kmeans':
        #print('Result of Kmeans')
        cur_clusters = nmt.get_clusters_kmeans_clustering(selectedUMAP, karg_dict)

    if method =='DBSCAN':
        #print('Result of DBSCAN')
        cur_clusters = nmt.get_clusters_dbscan_clustering(selectedUMAP, karg_dict)

    if method =='HDBSCAN':
        #print('Result of HDBSCAN')
        cur_clusters = nmt.get_clusters_hdbscan_clustering(selectedUMAP, karg_dict)
    selectedUMAP.loc[:,'Cluster'] = ['C' + str(i) for i in cur_clusters]
    return selectedUMAP

## function freq_Matrix(fre_M, uniqueDF, cluster_method, para_test, iternum)
* **fre_M** is the square matrix recording the number of co-clustering
* **uniqueDF** is the dataframe with unrepeated parameters
* **cluster_method** can be 'Hierarchy','Kmeans', 'DBSCAN','HDBSCAN','SNN_community'
* **para_test** is the input parameter dictionary for the cluster method
* **iternum** is the number of iteration to generate the coclustering matrix

In [ ]:
import random
import ast
from scipy.spatial.distance import pdist, squareform
import numpy as np
import matplotlib as mpl
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
def freq_Matrix(fre_M,uniqueDF,cluster_method, para_test,iternum):
    
    for i in range(iternum):
        umapDF = ns.UMAP.iloc[random.sample(range(0,ns.UMAP.shape[0]), int(ns.UMAP.shape[0]*0.95)),:].copy()
        resultDF = get_clusters(umapDF.copy(),method =cluster_method,karg_dict = para_test)
        Crange, Ccounts = np.unique(resultDF["Cluster"], return_counts = True)
        for iter_C in Crange:
            selected_row = resultDF[resultDF["Cluster"]==iter_C]
            Clist = selected_row.index.tolist()
            for sample_row in Clist:
                for sample_col in Clist:
                    fre_M.loc[sample_row,sample_col] =  fre_M.loc[sample_row,sample_col]+1
        print('****CODE RUNNING'+str(i/iternum)+' % ****')
    return fre_M.copy()

In [4]:
unique_hier = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_hier.xlsx', index_col=0)
unique_kmeans = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_kmeans.xlsx', index_col=0)
unique_dbscan = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_dbscan.xlsx', index_col=0)
unique_hdbscan = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_hdbscan.xlsx', index_col=0)
unique_snn = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_snn.xlsx', index_col=0)

### For Hierarchy

In [6]:

import random
import multiprocessing
import time
def hier_Matrix(fre_M):
    uniqueDF = unique_hier.copy()
    cluster_method = 'Hierarchy'
    para_test = par_hier
    umapDF = ns.UMAP.iloc[random.sample(range(0,ns.UMAP.shape[0]), int(ns.UMAP.shape[0]*0.95)),:].copy()
    resultDF = get_clusters(umapDF.copy(),method =cluster_method,karg_dict = para_test)
    Crange, Ccounts = np.unique(resultDF["Cluster"], return_counts = True)
    for iter_C in Crange:
        selected_row = resultDF[resultDF["Cluster"]==iter_C]
        Clist = selected_row.index.tolist()
        for sample_row in Clist:
            for sample_col in Clist:
                fre_M.loc[sample_row,sample_col] =  fre_M.loc[sample_row,sample_col]+1
    return fre_M.values
start = time.perf_counter ()
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
fre_M_t = pd.DataFrame(index = ns.UMAP.index, columns = ns.UMAP.index)
fre_M_t [fre_M_t.isnull()]=0

M_range= list()

for iter in range(1000):
   M_range.append(fre_M_t)

print('Length of M_range is '+str(len(M_range)))
# method 1: map
hier_paral_ap = sum(pool.map(hier_Matrix, M_range))
print(hier_paral_ap)  
elapsed = (time.perf_counter ()- start)
print(elapsed)





*************Note:**************
Method centroid requires the distance metric to be Euclidean

Length of M_range is 1000



*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean





*************Note:**************
Method centroid requires the distance metric to be Euclidean



*************Note:**************
Method centroid requires the distance metric to be Euclidean

















*************Note:**************
*************Note:**************
*************Note:**************



Method centroid requires the distance metric to be Euclidean
Method centroid requires the distance metric to be Euclidean

Method centroid requires the distance metric to be Euclidean
*************Note:**************
*************Note:**************
Method centroid r

Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requ





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean








*************Note:**************
*************Note:**************
Method centroid requires the distance metric to be Euclidean
Method centroid requires the distance metric to be Euclidean






*************Note:**************


Method centroid requires the distance metric to be Euclidean
*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Not





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Not





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Not

...





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Not





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean







*************Note:**************

Method centroid requires the distance metric to be Euclidean
*************Not





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Not





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Not





*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Note:**************
Method centroid requires the distance metric to be Euclidean




*************Not

In [ ]:
import seaborn as sns; sns.set()

ax=sns.clustermap(hier_paral_ap/1500)

In [ ]:
print(hier_paral_ap)  
elapsed = (time.clock() - start)
print(elapsed)

In [ ]:
import multiprocessing

def hier_Matrix(fre_M):
    uniqueDF = unique_hier.copy()
    cluster_method = 'Hierarchy'
    para_test = par_hier
    umapDF = ns.UMAP.iloc[random.sample(range(0,ns.UMAP.shape[0]), int(ns.UMAP.shape[0]*0.95)),:].copy()
    resultDF = get_clusters(umapDF.copy(),method =cluster_method,karg_dict = para_test)
    Crange, Ccounts = np.unique(resultDF["Cluster"], return_counts = True)
    for iter_C in Crange:
        selected_row = resultDF[resultDF["Cluster"]==iter_C]
        Clist = selected_row.index.tolist()
        for sample_row in Clist:
            for sample_col in Clist:
                fre_M.loc[sample_row,sample_col] =  fre_M.loc[sample_row,sample_col]+1
    return fre_M.values

cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
fre_M_t = pd.DataFrame(index = ns.UMAP.index, columns = ns.UMAP.index)
fre_M_t [fre_M_t.isnull()]=0
M_range= list()

for iter in range(20):
   M_range.append(fre_M_t)
ss=np.zeros(fre_M_t.shape)
for y in pool.imap(hier_Matrix, M_range):
    ss+=y

In [ ]:
# method 2: imap
for y in pool.imap(f, xs):
    print y            # 0, 1, 4, 9, 16, respectively

# method 3: imap_unordered
for y in pool.imap_unordered(f, xs):
    print(y)           # may be in any order

In [ ]:
fre_M_hdbscan = pd.DataFrame(index = ns.UMAP.index, columns = ns.UMAP.index)
fre_M_hdbscan [fre_M_hdbscan.isnull()]=0
fre_M_hdbscan = freq_Matrix(fre_M_hdbscan,unique_hdbscan,'HDBSCAN',par_hdbscan,5000)

In [ ]:
fre_M_hdbscan.to_excel('/home/penglab/FeaCal/dataSource/projFea/fre_M_hdbscan.xlsx')

SETTING CLUSTER NUMBER FROM 8 TO 40
#### Using soma location as features
### Hierarchy Clustering
For Hierarchy method, the most suitable parameter is {'L_method': 'complete', 'L_metric': 'braycurtis', 'criterionH': 'distance', 'depth': 2, 'R': None, 't': 0.9, 'optimal_ordering': False}
For Hierarchy method, the ARI is 0.3268728951610669
The setting cluster number's limit is satisfied, the final number of cluster is 9
### Kmeans Clustering
For Kmeans method, the most suitable parameter is {'n_clusters': 10, 'init': 'random', 'n_init': 7, 'max_iter': 300, 'tol': 0.0001, 'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 'algorithm': 'auto'}
For Kmeans method, the ARI is 0.18870810160144874
The setting cluster number's limit is satisfied, the final number of cluster is 10
### DBSCAN Clustering
For DBSCAN method, the most suitable parameter is {'eps': 0.9, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 'leaf_size': 30, 'p': None, 'n_jobs': None}
For DBSCAN method, the ARI is 0.2630886481583674
The setting cluster number's limit is satisfied, the final number of cluster is 29
### HDBSCAN Clustering
For HDBSCAN method, the most suitable parameter is {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.9, 'min_samples': 9, 'p': 2, 'algorithm': 'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 'prediction_data': False, 'match_reference_implementation': False}
For HDBSCAN method, the ARI is 0.26100195542904575
The setting cluster number's limit is satisfied, the final number of cluster is 34


### SNN Clustering
For SNN_community method, the most suitable parameter is {'knn': 5, 'metric': 'minkowski', 'method': 'FastGreedy'}
For SNN_community method, the ARI is 0.13064660013981225
The setting cluster number's limit is not satisfied, the final number of cluster is 65

In [ ]:
ns = nmt.neuron_set('/home/penglab/Documents/Janelia_1000')
_ = ns.ReduceDimUMAP(feature_set="soma_features", n_neighbors=100)

In [ ]:
par_hier2 =  {'L_method': 'complete', 'L_metric': 'braycurtis', 'criterionH': 'distance', 'depth': 2, 'R': None, 
             't': 0.9, 'optimal_ordering': False} 
par_kmeans2 = {'n_clusters': 10, 'init': 'random', 'n_init': 7, 'max_iter': 300, 'tol': 0.0001, 
              'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 
              'algorithm': 'auto'} 

par_dbscan2 = {'eps': 0.9, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 
              'leaf_size': 30, 'p': None, 'n_jobs': None}

par_hdbscan2 =  {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.9, 'min_samples': 9, 'p': 2, 'algorithm': 
                'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 
                'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 
                'prediction_data': False, 'match_reference_implementation': False} 

par_snn2 = {'knn':5,'metric':'minkowski','method':'FastGreedy'}


In [ ]:
fre_M_hdbscan2 = pd.DataFrame(index = ns.UMAP.index, columns = ns.UMAP.index)
fre_M_hdbscan2 [fre_M_hdbscan2.isnull()]=0
fre_M_hdbscan2 = freq_Matrix(fre_M_hdbscan2,unique_hdbscan,'DBSCAN',par_hdbscan2,5000)
fre_M_hdbscan2.to_excel('/home/penglab/FeaCal/dataSource/somaLoc/fre_M_hdbscan.xlsx')